In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import FloatSlider, interact, DatePicker
import datetime

### Constants

In [17]:
initial_supply = 1e9
max_supply = 1e10
decay_constant = 0.5
tge_date = datetime.datetime(2024, 1, 1)

In [18]:
# Dynamically adjusted inflation function
def dynamic_inflation(t, current_supply, max_supply, decay_constant):
    remaining_supply_ratio = (max_supply - current_supply) / max_supply
    inflation_rate = decay_constant * remaining_supply_ratio
    return inflation_rate

# Function to calculate supply based on dynamic inflation rate
def calculate_supply_dynamic(initial_supply, max_supply, decay_constant, time_horizon):
    supply = [initial_supply]
    monthly_mint = []
    for t in range(1, int(time_horizon) + 1):
        current_supply = supply[-1]
        inflation_rate = dynamic_inflation(t, current_supply, max_supply, decay_constant)
        new_supply = current_supply * (1 + inflation_rate / 12)  # Monthly compounding
        minted = new_supply - current_supply
        monthly_mint.append(minted)
        supply.append(min(new_supply, max_supply))
    return np.array(supply), monthly_mint

# Function to create the DataFrame
def create_emission_schedule_df(initial_supply, max_supply, decay_constant, time_horizon, tge_date):
    supply_dynamic, monthly_mint = calculate_supply_dynamic(initial_supply, max_supply, decay_constant, time_horizon)
    months = np.arange(1, len(supply_dynamic))
    dates = [tge_date + datetime.timedelta(days=30 * i) for i in range(len(months))]
    token_per_second = [mint / (30 * 24 * 60 * 60) for mint in monthly_mint]

    data = {
        'Total Minted': [initial_supply] + list(np.cumsum(monthly_mint)[:-1]),
        'Total per Month': monthly_mint,
        'Token/Second': token_per_second,
        'Date': [date.strftime("%Y-%m-%d") for date in dates]
    }
    df = pd.DataFrame(data, index=months, columns=['Total Minted', 'Total per Month', 'Token/Second', 'Date'])
    df.index.name = 'Month'
    df['Year'] = df.index // 12 + 1
    df.set_index(['Year', df.index], inplace=True)
    return df

# Styling the DataFrame
def style_dataframe(df):
    cm = sns.light_palette("green", as_cmap=True)
    styled_df = df.style.background_gradient(cmap=cm, subset=['Total Minted', 'Total per Month', 'Token/Second']) \
                     .format({'Total Minted': '{:,.0f}', 'Total per Month': '{:,.0f}', 'Token/Second': '{:.6f}'}) \
                     .set_properties(**{
                         'background-color': '#1f1f1f',
                         'color': '#ffffff',
                         'border-color': 'white'
                     })
    return styled_df

### Plotting the Total Supply

In [19]:
# Plotting the Total Supply
def plot_total_supply(df, decay_constant, time_horizon):
    supply_dynamic, _ = calculate_supply_dynamic(initial_supply, max_supply, decay_constant, time_horizon)
    months = np.arange(0, len(supply_dynamic))
    
    plt.figure(figsize=(14, 8))
    plt.plot(months, supply_dynamic, label='Total Supply', color='#2dff73')
    plt.axhline(y=max_supply, color='#39FF14', linestyle='--', label='Max Supply')
    plt.title('Total Supply of DGYM Over Time', color='#ffffff')
    plt.xlabel('Months', color='#ffffff')
    plt.ylabel('Total Supply', color='#ffffff')
    plt.xticks(color='#ffffff')
    plt.yticks(color='#ffffff')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.ylim(initial_supply, max_supply)
    plt.gca().set_facecolor('#000000')
    plt.show()

In [20]:
def interactive_plot(tge_date, decay_constant, time_horizon):
    df = create_emission_schedule_df(initial_supply, max_supply, decay_constant, time_horizon, tge_date)
    styled_df = style_dataframe(df)
    display(styled_df)
    plot_total_supply(df, decay_constant, time_horizon)

interact(interactive_plot,
         tge_date=DatePicker(value=tge_date, description='TGE Date'),
         decay_constant=FloatSlider(value=decay_constant, min=0.001, max=1.0, step=0.01, description='Decay Constant'),
         time_horizon=FloatSlider(value=180, min=1, max=180, step=1, description='Time Horizon (months)')
        );


interactive(children=(DatePicker(value=datetime.datetime(2024, 1, 1, 0, 0), description='TGE Date', step=1), F…